In [1]:
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import io

import pyspark.sql.functions as F
import pyspark

from pyspark.sql import SparkSession, Window
from pyspark.conf import SparkConf
# from pyspark.context import SparkContext
from pyspark.sql.types import StringType, ArrayType, StructField, StructType, FloatType, DoubleType, IntegerType

from concurrent.futures import ThreadPoolExecutor

from utilities.loaders import save_data_splits

%load_ext autoreload
%autoreload 2

In [2]:
# `sparksession is none: typeerror: 'javapackage' object is not 
# callable` can be raised if the pyspark version being used is 4.0.0
# which is not compatible to a python 3.11.8 version
spark = SparkSession.builder.appName("app")\
    .config("spark.driver.memory", "14g")\
    .config("spark.executor.memory", "2g")\
    .config("spark.executor.cores", "6")\
    .config("spark.sql.execution.arrow.maxRecordsPerBatch", "100")\
    .getOrCreate()

In [3]:
# URL = "abfss://{FOLDER_NAME}@sgppipelinesa.dfs.core.windows.net/"
# SILVER_FOLDER_NAME = "sgppipelinesa-silver"
# SUB_FOLDER_NAME = "stage-01"
# SILVER_DATA_PATH = os.path.join(URL.format(FOLDER_NAME=SILVER_FOLDER_NAME), SUB_FOLDER_NAME)
# SILVER_DATA_PATH
# folder_infos = dbutils.fs.ls(BRONZE_DATA_PATH)

DATA_PATH = "../include/data/"
SILVER_FOLDER_NAME = "silver"
SUB_FOLDER_NAME = "stage-01"
SILVER_DATA_PATH = os.path.join(DATA_PATH, os.path.join(SILVER_FOLDER_NAME, SUB_FOLDER_NAME))
SILVER_DATA_PATH

'../include/data/silver\\stage-01'

In [4]:
train_labels_df = spark.read.format("parquet").load(os.path.join(SILVER_DATA_PATH, "train", "labels.parquet"))
val_labels_df = spark.read.format("parquet").load(os.path.join(SILVER_DATA_PATH, "validate", "labels.parquet"))
test_labels_df = spark.read.format("parquet").load(os.path.join(SILVER_DATA_PATH, "test", "labels.parquet"))

In [5]:
train_labels_df.show()

+------+--------------------+--------------------+
| value|            filePath|           subjectId|
+------+--------------------+--------------------+
|  male|file:///c:/Users/...|23yipikaye-201008...|
|female|file:///c:/Users/...|Anniepoo-20140308...|
|female|file:///c:/Users/...|Anniepoo-20140308...|
|female|file:///c:/Users/...|Anniepoo-20140308...|
|female|file:///c:/Users/...|Anniepoo-20140308...|
|female|file:///c:/Users/...| 1337ad-20170321-tkg|
|  male|file:///c:/Users/...| 1snoke-20120412-hge|
|  male|file:///c:/Users/...|  Aaron-20080318-kdl|
|  male|file:///c:/Users/...|   1028-20100710-hne|
+------+--------------------+--------------------+



In [6]:
val_labels_df.show()

+------+--------------------+-------------------+
| value|            filePath|          subjectId|
+------+--------------------+-------------------+
|female|file:///c:/Users/...|1337ad-20170321-ajg|
|  male|file:///c:/Users/...| Coren-20141121-pxp|
+------+--------------------+-------------------+



In [7]:
test_labels_df.show()

+------+--------------------+--------------------+
| value|            filePath|           subjectId|
+------+--------------------+--------------------+
|female|file:///c:/Users/...|Anniepoo-20140308...|
+------+--------------------+--------------------+



#### we will load the data in this format for concurrent processing and to prevent bottle neck issues of having to read files into one dataframe and then just to only convert it back to a array fo tuples with subject name and a corresponding spark dataframe. So why not instead read the parquet files like this?
```
[
  (<subject 1>, <subject 1 spark df>),
  (<subject 2>, <subject 2 spark df>),
  (<subject 3>, <subject 3 spark df>),
  ...
  (<subject n>, <subject n spark df>),
]
```

In [8]:
def read_signal_files(SPLIT_FOLDER):

    # only include the parquet files without labels
    signal_files = [
        os.path.join(SPLIT_FOLDER, SIGNAL_DF_FILE).replace('\\', '/') for SIGNAL_DF_FILE in os.listdir(SPLIT_FOLDER) if not "labels" in SIGNAL_DF_FILE]

    def helper(signal_file):
        subject_id = signal_file.split('/')[-1].replace("_signals.parquet", "")
        signal_df = spark.read.format("parquet").load(signal_file)
        signal_df = signal_df.orderBy("ID")
        return subject_id, signal_df

    with ThreadPoolExecutor() as exe:
        signals_df = list(exe.map(helper, signal_files))

    return signals_df

In [9]:
SPLIT_URL = os.path.join(SILVER_DATA_PATH, "{SPLIT}")
SPLIT_FOLDER = SPLIT_URL.format(SPLIT="train")
train_signals_df = read_signal_files(SPLIT_FOLDER)

In [10]:
train_signals_df

[('1028-20100710-hne', DataFrame[subjectId: string, signals: float, ID: int]),
 ('1337ad-20170321-tkg',
  DataFrame[subjectId: string, signals: float, ID: int]),
 ('1snoke-20120412-hge',
  DataFrame[subjectId: string, signals: float, ID: int]),
 ('23yipikaye-20100807-ujm',
  DataFrame[subjectId: string, signals: float, ID: int]),
 ('Aaron-20080318-kdl', DataFrame[subjectId: string, signals: float, ID: int]),
 ('Anniepoo-20140308-bft',
  DataFrame[subjectId: string, signals: float, ID: int]),
 ('Anniepoo-20140308-cqj',
  DataFrame[subjectId: string, signals: float, ID: int]),
 ('Anniepoo-20140308-hns',
  DataFrame[subjectId: string, signals: float, ID: int]),
 ('Anniepoo-20140308-nky',
  DataFrame[subjectId: string, signals: float, ID: int])]

In [11]:
SPLIT_FOLDER = SPLIT_URL.format(SPLIT="validate")
val_signals_df = read_signal_files(SPLIT_FOLDER)

In [12]:
val_signals_df

[('1337ad-20170321-ajg',
  DataFrame[subjectId: string, signals: float, ID: int]),
 ('Coren-20141121-pxp', DataFrame[subjectId: string, signals: float, ID: int])]

In [13]:
SPLIT_FOLDER = SPLIT_URL.format(SPLIT="test")
test_signals_df = read_signal_files(SPLIT_FOLDER)

In [14]:
train_signals_df[0][-1].show()

+-----------------+-------------+---+
|        subjectId|      signals| ID|
+-----------------+-------------+---+
|1028-20100710-hne|-0.0054626465|  0|
|1028-20100710-hne|-0.0053710938|  1|
|1028-20100710-hne|-0.0055236816|  2|
|1028-20100710-hne| -0.005493164|  3|
|1028-20100710-hne| -0.005340576|  4|
|1028-20100710-hne|-0.0047302246|  5|
|1028-20100710-hne| -0.004486084|  6|
|1028-20100710-hne|-0.0047302246|  7|
|1028-20100710-hne|-0.0049438477|  8|
|1028-20100710-hne| -0.004760742|  9|
|1028-20100710-hne|-0.0042419434| 10|
|1028-20100710-hne| -0.004211426| 11|
|1028-20100710-hne| -0.004119873| 12|
|1028-20100710-hne|-0.0032348633| 13|
|1028-20100710-hne|-0.0029907227| 14|
|1028-20100710-hne|-0.0032958984| 15|
|1028-20100710-hne|-0.0035095215| 16|
|1028-20100710-hne|-0.0036621094| 17|
|1028-20100710-hne|-0.0036621094| 18|
|1028-20100710-hne| -0.003967285| 19|
+-----------------+-------------+---+
only showing top 20 rows



In [15]:
# Define a UDF to load audio with librosa
@F.udf(returnType=ArrayType(FloatType()))
def load_audio_with_librosa(content):
    if content is None:
        return None

    try:
        # Create a file-like object from the binary content 
        # which spark.read.format("binaryFile").load("<path>")
        # returns
        audio_buffer = io.BytesIO(content)

        # we convert this audio buffer array as audio using librosa
        # sr=None to preserve original sample rate
        y, sr = librosa.load(audio_buffer, sr=16000) 
        
        # top_db is set to 20 representing any signal below
        # 20 decibels will be considered silence
        y_trimmed, _ = librosa.effects.trim(y, top_db=20)

        # Convert numpy array to list for Spark dataframe
        return y_trimmed.tolist()
    
    except Exception as e:
        print(f"Error processing audio: {e}")
        return None


@F.pandas_udf(returnType=FloatType(), functionType=F.PandasUDFType.GROUPED_AGG)
def get_peak_freq(segment: pd.Series):
    # calculate frequency domain features
    # get the spectrogram by calculating short time fourier transform
    spectrogram = np.abs(librosa.stft(segment))
    # print(f"spectrogram shape: {spectrogram.shape}")

    # Get the frequencies corresponding to the spectrogram bins
    frequencies = librosa.fft_frequencies(sr=16000)
    # print(f"frequencies shape: {frequencies.shape}")

    # Find the frequency bin with the highest average energy
    peak_frequency_bin = np.argmax(np.mean(spectrogram, axis=1))

    # Get the peak frequency in Hz
    # calculate also peak frequency
    # I think dito na gagamit ng fast fourier transform
    # to obtain the frequency, or use some sort of function
    # to convert the raw audio signals into a spectogram
    peak_frequency = frequencies[peak_frequency_bin]

    return peak_frequency

def extract_features(dataset: list[tuple[str, pyspark.sql.DataFrame]], hertz: int=16000, window_time: int=3, hop_time: int=1):
    """
    extracts the features from each segment of an audio signal

    args:
        dataset - 
        hertz - number of samples per second
        window_time - number of seconds of the given window to consider
        e.g. if number of seconds is 3 and hertz is 16000 or 16000
        samples/rows per second then the window size we will consider
        is 16000 * 3 or 48000
        hop_time - seconds
    """
    
    def helper(datum):
        # we access the SCR values via raw data column
        subject_id = datum[0]
        signal_df = datum[1]

        # # get number of rows of 16000hz signals 
        # n_rows = x_signals.shape[0]
        # # print(n_rows)

        # we calculate the window size of each segment or the
        # amount of samples it has to have based on the frequency
        samples_per_win_size = int(window_time * hertz)
        samples_per_hop_size = int(hop_time * hertz)
        # print(f"samples per window size: {samples_per_win_size}")
        # print(f"samples per hop size: {samples_per_hop_size}\n")

        
        feat_window = Window.orderBy("ID").rowsBetween(Window.currentRow, samples_per_win_size - 1)
        
        signal_df = signal_df.withColumn("freq_skew", F.skewness("signals").over(feat_window))
        signal_df = signal_df.withColumn("freq_kurt", F.kurtosis("signals").over(feat_window))
        
        signal_df = signal_df.withColumn("freq_mean", F.mean("signals").over(feat_window))
        # signal_df = signal_df.withColumn("freq_median", F.median("signals").over(feat_window))
        # median over window function is not supported so we can use 
        signal_df = signal_df.withColumn("freq_median", F.percentile("signals", 0.5).over(feat_window))
        signal_df = signal_df.withColumn("freq_mode", F.mode("signals").over(feat_window))
        
        signal_df = signal_df.withColumn("freq_min", F.min("signals").over(feat_window))
        signal_df = signal_df.withColumn("freq_max", F.max("signals").over(feat_window))
        signal_df = signal_df.withColumn("freq_range", F.col("freq_max") - F.col("freq_min"))
        signal_df = signal_df.withColumn("freq_var", F.variance("signals").over(feat_window))
        signal_df = signal_df.withColumn("freq_std", F.stddev("signals").over(feat_window))
        
        signal_df = signal_df.withColumn("freq_first_quart", F.percentile("signals", 0.25).over(feat_window))
        signal_df = signal_df.withColumn("freq_third_quart", F.percentile("signals", 0.75).over(feat_window))
        signal_df = signal_df.withColumn("freq_inter_quart_range", F.col("freq_first_quart") - F.col("freq_third_quart"))

        # signal_df = signal_df.withColumn("freq_peak", get_peak_freq(F.col("signals")).over(feat_window))
        
        # an implementation of the only including windows after a certain
        # hop size, since we cannot do it directly using spark we can 
        # filter out the rows of windows that have not yet made the 
        # appropriate hop size using filtering 
        signal_df = signal_df.where((F.col("ID") % samples_per_hop_size) == 0)

        # # initialize segments to empty list as this will store our
        # # segmented signals 
        # segments = []
        # labels = []

        # # fig = plt.figure(figsize=(17, 5))
        # n_frames = 0

        # # this segments our signals into overlapping segments
        # for i in range(0, (n_rows - samples_per_win_size) + samples_per_hop_size, samples_per_hop_size):
        #     # # last segment would have start x: 464000 - end x: 512000
        #     # # and because 512000 plus our hop size of 16000 = 528000 
        #     # # already exceeding 521216 this then terminates the loop
        #     # i += samples_per_hop_size
        #     # start = i
        #     # end = i + samples_per_win_size
        #     start = i
        #     end = min((i + samples_per_win_size), n_rows)
        #     # print(f'start x: {start} - end x: {end}')

        #     # extract segment from calculated start and end
        #     # indeces
        #     segment = x_signals[start:end]

        #     # # calculate frequency domain features
        #     # # get the spectrogram by calculating short time fourier transform
        #     # spectrogram = np.abs(librosa.stft(segment))
        #     # # print(f"spectrogram shape: {spectrogram.shape}")

        #     # # Get the frequencies corresponding to the spectrogram bins
        #     # frequencies = librosa.fft_frequencies(sr=hertz)
        #     # # print(f"frequencies shape: {frequencies.shape}")

        #     # # Find the frequency bin with the highest average energy
        #     # peak_frequency_bin = np.argmax(np.mean(spectrogram, axis=1))

        #     # # Get the peak frequency in Hz
        #     # # calculate also peak frequency
        #     # # I think dito na gagamit ng fast fourier transform
        #     # # to obtain the frequency, or use some sort of function
        #     # # to convert the raw audio signals into a spectogram
        #     # peak_frequency = frequencies[peak_frequency_bin]

        #     # # calculate the segments fast fourier transform
        #     # ft = np.fft.fft(segment)

        #     # # the fft vector can have negative or positive values
        #     # # so to avoid negative values and just truly see the frequencies
        #     # # of each segment we use its absolute values instead 
        #     # magnitude = np.abs(ft)
        #     # mag_len = magnitude.shape[0]
        #     # frequency = np.linspace(0, hertz, mag_len)

        #     # calculate statistical features
        #     # because the frequency for each segment is 16000hz we can divide
        #     # it by 1000 to instead to get its kilo hertz alternative
        #     mean_freq_kHz = np.mean(segment, axis=0)
        #     median_freq_kHz = np.median(segment, axis=0)
        #     std_freq = np.std(segment, axis=0)
        #     mode_freq = mode(segment, axis=0)
            
        #     # min = np.min(segment, axis=0)

        #     # calculate first quantile, third quantile, interquartile range
        #     first_quartile_kHz = np.percentile(segment, 25) / 1000,
        #     third_quartile_kHz = np.percentile(segment, 75) / 1000,
        #     inter_quartile_range_kHz = (np.percentile(segment, 75) - np.percentile(segment, 25)) / 1000,

        #     # compute morphological features
        #     skewness = skew(segment)
        #     kurtosis = kurt(segment)

        #     # compute time domain features
        #     amp_env = np.max(segment, axis=0)
        #     rms = np.sqrt(np.sum(segment ** 2, axis=0) / samples_per_win_size)

        #     features = {
        #         # statistical features
        #         "mean_freq_kHz": mean_freq_kHz,
        #         "median_freq_kHz": median_freq_kHz,
        #         "std_freq": std_freq,
        #         "mode_freq": mode_freq[0],
        #         'first_quartile_kHz': first_quartile_kHz[0],
        #         'third_quartile_kHz': third_quartile_kHz[0],
        #         'inter_quartile_range_kHz': inter_quartile_range_kHz[0],

        #         # morphological features
        #         "skewness": skewness,
        #         "kurtosis": kurtosis,

        #         # time domain features
        #         "amp_env":amp_env,
        #         "rms": rms,
                
        #         # frequency features
        #         # "peak_frequency": peak_frequency,
        #     }
            
        #     segments.append(features)
        #     labels.append(label)
            
        #     n_frames += 1

        # frames = range(n_frames)
        # # print(f"number of frames resulting from window size of {samples_per_win_size} and a hop size of {samples_per_hop_size} from audio signal frequency of {hertz}: {frames}")

        # time = librosa.frames_to_time(frames, hop_length=samples_per_hop_size)
        # # print(f"shape of time calculated from number of frames: {time.shape[0]}\n")
        
        # # calculate other features
        # zcr = librosa.feature.zero_crossing_rate(y=x_signals, frame_length=samples_per_win_size, hop_length=samples_per_hop_size)
        # mel_spect = librosa.feature.melspectrogram(y=x_signals, sr=hertz, n_fft=samples_per_win_size, hop_length=samples_per_hop_size, n_mels=90)
        # mel_spect_db = librosa.power_to_db(mel_spect, ref=np.max)
        # mean_mel = np.mean(mel_spect_db, axis=0)
        # variance_mel = np.var(mel_spect_db, axis=0)

        # spect_cent = librosa.feature.spectral_centroid(y=x_signals, sr=hertz, n_fft=samples_per_win_size, hop_length=samples_per_hop_size)
        # # chroma_stft = librosa.feature.chroma_stft(y=x_signals, frame_length=samples_per_win_size, hop_length=samples_per_hop_size)
        # # print(mel_spect.shape, spect_cent.shape, zcr.shape)
        # # print(f"mel spectrogram shape: {mel_spect.shape}")

        # # calculate the number of values we need to remove in the
        # # feature vector librosa calculated for us compared to the
        # # feature vectors we calculated on our own
        # zcr_n_values_to_rem = np.abs(zcr.shape[1] - time.shape[0])
        # mean_mel_n_values_to_rem = np.abs(mean_mel.shape[0] - time.shape[0])
        # spect_cent_n_values_to_rem = np.abs(spect_cent.shape[1] - time.shape[0])

        # # get slice of those in range with time only
        # zcr = zcr.reshape(-1)[:-zcr_n_values_to_rem]
        # mean_mel = mean_mel.reshape(-1)[:-mean_mel_n_values_to_rem]
        # variance_mel = variance_mel.reshape(-1)[:-mean_mel_n_values_to_rem]
        # spect_cent = spect_cent.reshape(-1)[:-spect_cent_n_values_to_rem]

        # # create features dataframe
        # subject_features = pd.DataFrame.from_records(segments)
        # subject_features["zcr"] = zcr
        # subject_features["mean_mel"] = mean_mel
        # subject_features["variance_mel"] = variance_mel
        # subject_features["spect_cent"] = spect_cent
        
        # # create labels dataframe
        # subject_labels = pd.Series(labels)

        # os.makedirs(f"./data/_EXTRACTED_FEATURES/{split}", exist_ok=True)
        # subject_features.to_csv(f'./data/_EXTRACTED_FEATURES/{split}/{name}_features.csv')
        # subject_labels.to_csv(f'./data/_EXTRACTED_FEATURES/{split}/{name}_labels.csv')

        # return (subject_features, subject_labels, name, time)
        return subject_id, signal_df

    with ThreadPoolExecutor(max_workers=2) as exe: 
        signals_df = list(exe.map(helper, dataset))

        # # unzip subjects data and unpack
        # subjects_features, subjects_labels, subjects_names, time = zip(*subjects_data)
    
    # return subjects_features, subjects_labels, subjects_names, time
    return signals_df

c:\Users\LARRY\anaconda3\envs\tech-interview\Lib\site-packages\pyspark\sql\pandas\functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [16]:
train_signals_df[:1]

[('1028-20100710-hne', DataFrame[subjectId: string, signals: float, ID: int])]

In [17]:
train_signals_df = extract_features(dataset=train_signals_df[:1])

In [18]:
train_signals_df

[('1028-20100710-hne',
  DataFrame[subjectId: string, signals: float, ID: int, freq_skew: double, freq_kurt: double, freq_mean: double, freq_median: double, freq_mode: float, freq_min: float, freq_max: float, freq_range: float, freq_var: double, freq_std: double, freq_first_quart: double, freq_third_quart: double, freq_inter_quart_range: double])]

In [19]:
train_signals_df[0][-1]

DataFrame[subjectId: string, signals: float, ID: int, freq_skew: double, freq_kurt: double, freq_mean: double, freq_median: double, freq_mode: float, freq_min: float, freq_max: float, freq_range: float, freq_var: double, freq_std: double, freq_first_quart: double, freq_third_quart: double, freq_inter_quart_range: double]

In [20]:
# train_signals_df[0][-1].select("freq_min", "ID").show()

In [21]:
# SILVER_FOLDER_NAME = "sgppipelinesa-silver"
# SUB_FOLDER_NAME = "stage-02"
# SILVER_DATA_PATH = URL.format(FOLDER_NAME=os.path.join(SILVER_FOLDER_NAME, SUB_FOLDER_NAME))
# SILVER_DATA_PATH
SILVER_FOLDER_NAME = "silver"
SUB_FOLDER_NAME = "stage-02"
SILVER_DATA_PATH = os.path.join(DATA_PATH, os.path.join(SILVER_FOLDER_NAME, SUB_FOLDER_NAME))
SILVER_DATA_PATH

'../include/data/silver\\stage-02'

In [ ]:
save_data_splits(train_signals_df, split="train", type_="signals", save_path=SILVER_DATA_PATH)
# save_data_splits(val_signals_df, split="validate", type_="signals", save_path=SILVER_DATA_PATH)
# save_data_splits(test_signals_df, split="test", type_="signals", save_path=SILVER_DATA_PATH)

saving 1028-20100710-hne signals...


In [ ]:
sample_data = [(i, (i + 1) * 10) for i in range(20)]
df = spark.createDataFrame(sample_data, ["row_id", "value"])

In [ ]:
df.show()

+------+-----+
|row_id|value|
+------+-----+
|     0|   10|
|     1|   20|
|     2|   30|
|     3|   40|
|     4|   50|
|     5|   60|
|     6|   70|
|     7|   80|
|     8|   90|
|     9|  100|
|    10|  110|
|    11|  120|
|    12|  130|
|    13|  140|
|    14|  150|
|    15|  160|
|    16|  170|
|    17|  180|
|    18|  190|
|    19|  200|
+------+-----+



In [ ]:
samples_per_win_size = 6
samples_per_hop_size = 4

In [ ]:
feat_window = Window.orderBy("row_id").rowsBetween(Window.currentRow, samples_per_win_size - 1)

In [ ]:
df = df.withColumn("freq_std", F.sum("value").over(feat_window))
df.show()

+------+-----+--------+
|row_id|value|freq_std|
+------+-----+--------+
|     0|   10|     210|
|     1|   20|     270|
|     2|   30|     330|
|     3|   40|     390|
|     4|   50|     450|
|     5|   60|     510|
|     6|   70|     570|
|     7|   80|     630|
|     8|   90|     690|
|     9|  100|     750|
|    10|  110|     810|
|    11|  120|     870|
|    12|  130|     930|
|    13|  140|     990|
|    14|  150|    1050|
|    15|  160|     900|
|    16|  170|     740|
|    17|  180|     570|
|    18|  190|     390|
|    19|  200|     200|
+------+-----+--------+



# an implementation of the only including windows after a certain hop size, since we cannot do it directly using spark we can filter out the rows of windows that have not yet made the appropriate hop size using filtering 

In [ ]:
cond = ((F.col("row_id") % samples_per_hop_size) == 0)
df.where(cond).show()

+------+-----+--------+
|row_id|value|freq_std|
+------+-----+--------+
|     0|   10|     210|
|     4|   50|     450|
|     8|   90|     690|
|    12|  130|     930|
|    16|  170|     740|
+------+-----+--------+

